In [9]:
from datetime import datetime
import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Choose your destiny: 'mainnet' or 'testnet'
network = 'mainnet'  # or 'testnet'
endpoints = {
    'mainnet': 'https://api.poktscan.com/poktscan/api/graphql',
    'testnet': 'https://api.testnet.poktscan.com/poktscan/api/graphql'
}

api_key = str(open("../../POKT Scan.txt", "r").read())

# Set up the headers with the provided API key and content type
headers = {
    "Authorization": api_key,
    "Content-Type": "application/json"
}

# Set up the transport with the chosen GraphQL endpoint and your headers
transport = RequestsHTTPTransport(
    url=endpoints[network],
    headers=headers,
    use_json=True
)

# Create the client
client = Client(transport=transport, fetch_schema_from_transport=True)

# Define your query here
query = gql("""
query {
  GetTopAccountsByBalance(
    input: {
      top: 1000
    }
  ) {
    point
    balance
    percentage
  }
}
""")

try:   
    # Execute the query and get the results
    response = client.execute(query)

    # Convert the result to a pandas DataFrame
    df = pd.DataFrame(response['GetTopAccountsByBalance'])

    # Generate a timestamp and integrate it into the filename
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    #filename = f"top_accounts_{current_time}.csv"

    # Save the DataFrame to a CSV
    #df.to_csv(filename, index=False)
    
except Exception as e:
    print(f"An error occurred: {e}")

In [10]:
df

,point,balance,percentage
0,f3fdda68d3e854600f72f90a66ca8c362effc32f,77586759291200,0.098921
1,ab04976668dd739f34df43e912d20dc487ac85d2,37550609047857,0.047876
2,26a0f23681bc77ab46c55d0b3d08aa5a5b43e1eb,35982439345061,0.045877
3,2eb054616797de8565506333afb334655e7774ed,32058484917314,0.040874
4,b400d3114247d55cdb57034028aa2590d4fc112a,23646703053355,0.030149
...,...,...,...
996,32b3ee43260912207704c79bda0229711d8a18d3,39490697730,0.000050
997,344e7bd9fc60a7f91f91b44219a5e7ef99af9810,39489860618,0.000050
998,3f56d024c74ede0d7cdfad35c2472d47a74b0a35,39466251920,0.000050
999,29d42fce518dfc79d2b5af0eb00bd3aab6c80f33,39434230000,0.000050


In [ ]:

​

​

​
# Set up the transport with the chosen GraphQL endpoint and your headers
transport = RequestsHTTPTransport(
    url=endpoints[network],
    headers=headers,
    use_json=True
)
​
# Create the client
client = Client(transport=transport, fetch_schema_from_transport=True)
​
# Define your query here
query = gql("""
query {
  GetTopAccountsByBalance(
    input: {
      top: 1000
    }
  ) {
    point
    balance
    percentage
  }
}
​
""")
​
try:   
    # Execute the query and get the results
    response = client.execute(query)
​
    # Convert the result to a pandas DataFrame
    df = pd.DataFrame(response['GetTopAccountsByBalance'])
​
    # Generate a timestamp and integrate it into the filename
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"top_accounts_{current_time}.csv"
​
    # Save the DataFrame to a CSV
    df.to_csv(filename, index=False)
    
except Exception as e:
    print(f"An error occurred: {e}")